# Spark Consumer

In [ ]:
from __future__ import print_function

import sys, os
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, split, from_json, flatten, udf

from config import *

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1 pyspark-shell' 

In [ ]:
print(len(tweet_keys))
print(len(tweet_types))
print(len(user_keys))
print(len(user_types))

In [ ]:
#spark.stop()

In [ ]:
spark = SparkSession\
    .builder\
    .appName("StructuredTwitterJsonArchive")\
    .getOrCreate()

In [ ]:
def assembleStructType(ST,keys,types):
    ST = ST.add(keys[0],types[0]())
    if len(keys)>1:
        return assembleStructType(ST,keys[1:],types[1:])
    else:
        return ST

test = assembleStructType(StructType(), tweet_keys, tweet_types)



In [ ]:
# value schema: { "user": "user_keys", "tweet": "tweet_keys" }

tweet_struct = assembleStructType(StructType(), tweet_keys, tweet_types)

user_struct = assembleStructType(StructType(), user_keys, user_types)

schema = StructType().add("users", user_struct).add("tweets", tweet_struct)

print(schema)

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic) \
  .option("failOnDataLoss" , "false")\
  .load()\
  .select(col("key").cast("string"), from_json(col("value").cast("string"), schema).alias('value'))


## Raw JSON Data to archive 

In [ ]:
# Json to HDFS sink with partitioning

targetJsonHDFS = df\
    .select(col('key'),col('value.*'))\
    .writeStream\
    .format("json")\
    .outputMode("append")\
    .partitionBy("key")\
    .option("path", "hdfs://localhost:8020" + hdfs_archive_path)\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_archive_path + "/sparkcheckpoint")\
    .start()
#targetJsonHDFS.awaitTermination()

In [ ]:
poll_continiously(targetJsonHDFS)

In [ ]:
targetJsonHDFS.explain()

In [ ]:
targetJsonHDFS.exception()

In [ ]:
targetJsonHDFS.stop()

## Tweets to HIVE

In [ ]:

def Sentiment(sent):
    from textblob import TextBlob
    return int(TextBlob(sent).sentiment.polarity*100)

Sentiment_udf = udf(Sentiment, IntegerType())


In [ ]:
import nltk
nltk.download('punkt')

In [ ]:

def n_words(sent):
    from textblob import TextBlob
    return len(TextBlob(sent).words)
n_words("hello how are you")

n_words_udf = udf(n_words, IntegerType())


In [ ]:
# Dump new tweets to HDFS as Parquet (small files):
tweets2HIVE = df\
    .select(col('key'),col(f'value.tweets.*'))\
    .withColumn('Sentiment',Sentiment_udf('text'))\
    .withColumn('n_words',n_words_udf('text'))\
    .where("n_words > 10")\
    .writeStream\
    .format("parquet")\
    .outputMode("append")\
    .partitionBy("key")\
    .trigger(processingTime="1 minutes")\
    .option("path", "hdfs://localhost:8020" + hdfs_hive_tweets)\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_hive_tweets + "/sparkcheckpoint" )\
    .start()   

In [ ]:
poll_continiously(tweets2HIVE)

In [ ]:
tweets2HIVE.explain()

In [ ]:
tweets2HIVE.exception()

In [ ]:
tweets2HIVE.stop()

## Users to HIVE

In [ ]:
# Dump new users to HDFS as Parquet (small files): 
users2HIVE = df\
    .select(col('key').alias('last_tweet_at'), col('value.users.*'))\
    .dropDuplicates(subset=['id'])\
    .writeStream\
    .format("parquet")\
    .outputMode("append")\
    .partitionBy(partitionCol)\
    .option("path", "hdfs://localhost:8020" + hdfs_hive_users_staging)\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_hive_users_staging + "/sparkcheckpoint" )\
    .start()   

In [ ]:
poll_continiously(users2HIVE)

In [ ]:
users2HIVE.explain()

In [ ]:
users2HIVE.exception()

In [ ]:
users2HIVE.stop()

## HIVE refresher job for tweets and users

In [ ]:
def refresh_hive_table(df, epoch_id):
    from pyhive import hive
    hive_cnx = hive.Connection(
        host = 'localhost', 
        port = 10000, 
        username = 'hdfs',
        password = 'naya',
        auth = 'CUSTOM',
        database=hive_database)
    with hive_cnx.cursor() as cursor:    
        cursor.execute(" MSCK REPAIR TABLE tweets ")      
        cursor.execute(" MSCK REPAIR TABLE users ")


In [ ]:
HIVErefresher = df\
                .writeStream\
                .foreachBatch(refresh_hive_table)\
                .trigger( processingTime='1 minute')\
                .start()

In [ ]:
poll_continiously(HIVErefresher, period=.2)

In [ ]:
HIVErefresher.status

In [ ]:
HIVErefresher.stop()

# Make users unique by 4-step aggregation query (does not work)

In [ ]:
       

def assemble_queries(year):
    queries = []

    queries.append("SET hive.exec.dynamic.partition.mode=nonstrict")
    queries.append("SET hive.exec.dynamic.partition = true;")
    queries.append("SET hive.mapred.mode = nonstrict;")
    queries.append("SET hive.optimize.sort.dynamic.partition=false")
    queries.append("SET spark.sql.hive.convertMetastoreParquet=false")
    #queries.append("SET hive.exec.stagingdir=.hive-staging;")
    queries.append("MSCK REPAIR TABLE users_staging")

    cols = ', '.join(colNames)

    queries.append(f"insert into users_staging partition({partitionCol}={year}) select {cols} from users where {partitionCol}={year}")

    #queries.append(f"alter table users drop partition ({partitionCol}={year})")

    #This is probably the right query for the 'merge' of partitions
    q42 = f"""
    SELECT last_tweet_at, id, name, screen_name, created_at, `location` , url, protected, verified, followers_count, friends_count, 
           listed_count, favourites_count, statuses_count, withheld_in_countries
    FROM (
        SELECT last_tweet_at, id, name, screen_name, created_at, `location` , url, protected, verified, followers_count, friends_count, 
           listed_count, favourites_count, statuses_count, withheld_in_countries,
           rank() over (partition by id order by last_tweet_at desc) as last_rank
        from users_staging
        where {partitionCol} ={year}
    ) ranked_users
    WHERE ranked_users.last_rank = 1
    """

    queries.append(f"insert overwrite table users partition({partitionCol}={year}) {q42}")

    queries.append(f"alter table users_staging drop partition ({partitionCol}={year})")
    return queries

assemble_queries('201703')

In [ ]:
#q = "select distinct created_ym from users_staging order by created_ym desc"
queries = ["MSCK REPAIR TABLE users_staging"]
queries.append("select created_ym, count(created_ym) from users_staging group by created_ym")
res = run_queries(queries)

years = [r[0] for r in res]
years = cycle(years)
years

In [ ]:
errors = []

In [ ]:
# while True:    
#     year = next(years)
#     print('|' + year , end = '')
#     try:
#         queries = assemble_queries(year)
#         run_queries(queries)
#     except KeyboardInterrupt:
#         print('KeyboardInterrupt')
#         break    
#     except Exception as e :
#         errors.append([year,e])
#         print("oops")
    

In [ ]:
errors

In [ ]:
year = '201703'

queries = []

queries.append("SET hive.exec.dynamic.partition.mode=nonstrict")
queries.append("SET hive.exec.dynamic.partition = true;")
queries.append("SET hive.mapred.mode = nonstrict;")
queries.append("SET hive.optimize.sort.dynamic.partition=false")
queries.append("SET spark.sql.hive.convertMetastoreParquet=false")
queries.append("SET hive.exec.stagingdir=.hive-staging;")
queries.append("MSCK REPAIR TABLE users_staging")

cols = ', '.join(colNames)

queries.append(f"insert into users_staging partition({partitionCol}) select {cols} from users")

#queries.append(f"alter table users drop partition ({partitionCol}={year})")

#This is probably the right query for the 'merge' of partitions
q42 = f"""
SELECT last_tweet_at, id, name, screen_name, created_at, `location` , url, protected, verified, followers_count, friends_count, 
       listed_count, favourites_count, statuses_count, withheld_in_countries, created_ym 
FROM (
    SELECT last_tweet_at, id, name, screen_name, created_at, `location` , url, protected, verified, followers_count, friends_count, 
       listed_count, favourites_count, statuses_count, withheld_in_countries, created_ym, 
       rank() over (partition by id order by last_tweet_at desc) as last_rank
    from users_staging        
) ranked_users
WHERE ranked_users.last_rank = 1
"""

queries.append(f"insert overwrite table users partition({partitionCol}) {q42}")

#queries.append(f"alter table users_staging drop partition ({partitionCol}={year})")

print(queries[-1])

